# Aviso importante!

O objetivo principal desse projeto é apenas mostrar que é possível utilizar machine learning para auxiliar o investidor na sua tomada de decisão.

Esse projeto não é uma indicação de investimento! Toda estratégia e resultados apresentados **são apenas para fins didáticos.**

In [ ]:
#instalação inicial das bibliotecas
!pip install yfinance
!pip install investpy
!pip install pandas
!pip install matplotlib
!pip install plotly
!pip install seaborn
!pip install numpy

# 1. Importação de bibliotecas

In [ ]:
import yfinance
import investpy

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler

from sklearn import linear_model
from statsmodels.tsa.arima_model import ARIMA
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score

# 2. Coleta dos dados

Nessa etapa será utilizado o método **download()** da biblioteca **yfinance** para obter os dados de determinada ação.

Esse método retorna um **dataframe** com as devidas colunas referentes à negociação feita em cada dia em que a ação foi negociada na bolsa de valores.

A priori, serão utilizados os dados da empresa ITAU mas outras podem ser analisadas, basta alterar o ticker da ação.

Ticker analisado: ITUB4.SA

In [ ]:
#Obtendo dataset da ação (01/01/2016 até 30/06/21) 
ticker = "ITUB4.SA"
df = yfinance.download(ticker, start="2016-01-01", end="2021-06-30")

In [ ]:
#Obtendo dataset do dolar (01/01/2016 até 30/06/21) 
dolar = investpy.get_currency_cross_historical_data(currency_cross='USD/BRL', from_date='01/01/2016', to_date='30/06/2021')

In [ ]:
df.dtypes

In [ ]:
dolar.dtypes

# 3. Tratamento dos dados

## 3.1. Verificando dataset da ação

In [ ]:
df.describe()

In [ ]:
df.tail()

In [ ]:
# quantidade de registros do dataset da ação
df.count()

In [ ]:
# identificando dados faltantes
df.isnull().sum()

## 3.2. Verificando dataset dolar

In [ ]:
dolar.describe()

In [ ]:
dolar.head()

In [ ]:
# quantidade de registros do dataset da dólar
dolar.count()

In [ ]:
# identificando dados faltantes
dolar.isnull().sum()

## 3.3. Integrando os datasets

Ao importar os dados das ações e do dólar possuem o campo Date como index. Utilizamos o **reset_index()** para modificar o index do dataset sem excluir a coluna (**drop=False**)

In [ ]:
#necessário fazer o reset_index pois o dataset vem indexado pela data
df = df.reset_index(drop=False)
df.head()

In [ ]:
dolar = dolar.reset_index(drop=False)
dolar.head()

In [ ]:
df.dtypes

In [ ]:
dolar.dtypes

**Juntando os dois datasets**

Como o objetivo é realizar uma tarefa de predição atributos não numéricos não serão utilizados. Dessa forma, até para poupar processamento, a coluna **Currency** será eliminada do dataset do dólar.

Nessa etapa será realizado a integração dos dois datasets em apenas um. Para isso modificamos o nome dos atributos do dólar para facilitar a identificação no dataset integrado.

In [ ]:
dolar = dolar.drop('Currency', axis=1)

In [ ]:
df_dolar = dolar.rename(columns={"Open": "Open_dolar",
                                 "High": "High_dolar",
                                 "Low": "Low_dolar",
                                 "Close": "Close_dolar"})
df_dolar.head()

In [ ]:
df_full = pd.merge(df, df_dolar, on='Date', how='inner')
df_full.head()

In [ ]:
# tamanho do dataset integrado
df_full.count()

In [ ]:
# verificando o último registro no dataset integrado (29/06/2021).
df_full.tail(5)

Importante registrar em uma outra variável o último registro, no caso o dia 29/06/2021, pois após treinar os modelos tentaremos prever o valor do Adj Close desse dia.

In [ ]:
ultimo_registro = df_full.tail(1).copy()

# 4. Visualização de Dados

In [ ]:
# gráfico de candlestick para ação
# criamos uma variável, do tipo dicionário (chave e valor), para personalizar a lista de impressão no gráfico.
trace1 = {
    'x': df_full.Date,
    'open': df_full.Open,
    'close': df_full.Close,
    'high': df_full.High,
    'low': df_full.Low,
    'type': 'candlestick',
    'name': 'ITUB4.SA',
    'showlegend': True
}

data = [trace1]
layout = go.Layout()

fig1 = go.Figure(data=data, layout=layout)
fig1.show()

In [ ]:
# gráfico de candlestick do dolar
trace2 = {
    'x': df_full.Date,
    'open': df_full.Open_dolar,
    'close': df_full.Close_dolar,
    'high': df_full.High_dolar,
    'low': df_full.Low_dolar,
    'type': 'candlestick',
    'name': 'Dólar',
    'showlegend': True
}

data = [trace2]
layout = go.Layout()

fig2 = go.Figure(data=data, layout=layout)
fig2.show()

In [ ]:
sns.heatmap(df_full.corr(), annot = False) # O parâmetro annot = True faz com que os valores fiquem visíveis no mapa.
plt.show()

In [ ]:
plt.figure(figsize=(16,6)) #altura e largura do gráfico
plt.plot(df_full['Date'], df_full['Adj Close'])
plt.plot(df_full['Date'], df_full['Close_dolar'])
plt.legend(['Adj Close', 'Close_dolar'])
plt.grid()
plt.title("Cotação da ação e dólar x Tempo", fontsize = 20)
plt.show()

# 5. Machine Learning

## 5.1. Preparando dataset para os algoritmos de ML

In [ ]:
# Deslocando os valores da coluna 'Adj Close' para cima utilizando o método shift(-1).
# isso é necessário para verificar a previsão do próximo dia, utilizando as features ou características do dia atual.

df_full['Adj Close'] = df_full['Adj Close'].shift(-1)

In [ ]:
#verificando novo valor do dia 29-06-2021 (deve ser NaN)
df_full.tail()

In [ ]:
df_full = df_full.dropna()
df_full.isnull().sum()

In [ ]:
df_full.tail()

In [ ]:
ultimo_registro

## 5.2. Selecionando features com KBest

In [ ]:
features = df_full.drop(['Date','Adj Close'],1)
label = df_full['Adj Close']

In [ ]:
features.head()

In [ ]:
features_list = ('Open','High','Low','Close','Volume', 'Open_dolar','High_dolar','Low_dolar','Close_dolar')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, label)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()
print ('')
print ("Melhores features:")
print (k_best_features_final)

In [ ]:
#separando as features escolhidas
features = df_full.loc[:,['Low_dolar', 'Close_dolar', 'Close','Volume', 'High_dolar']]

In [ ]:
features

## 5.3. Separando o dataset para treinamento e teste

In [ ]:
qtd_linhas = len(df_full)

qtd_linhas_treino= round(.70 * qtd_linhas)
qtd_linhas_teste= qtd_linhas - qtd_linhas_treino  

print("Quant linhas de treino: 0:"+ str(qtd_linhas_treino))
print("Quant linhas de teste: " + str(qtd_linhas_treino) + ":" + str(qtd_linhas_treino + qtd_linhas_teste -1))

In [ ]:
#Separa os dados de treino e teste
X_train = features[:qtd_linhas_treino]
X_test = features[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

y_train = label[:qtd_linhas_treino]
y_test = label[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

print(len(X_train), len(y_train))

print(len(X_test), len(y_test))

# 5.4. Normalizando os dados

Uma importante etapa é o processo de normalização dos dados, para que o treinamento dos modelos sejam feitos com todas as features (características/colunas) dentro de uma mesma escala.

In [ ]:
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)  # Normalizando os dados de entrada(treinamento)
X_test_scale  = scaler.transform(X_test)       # Normalizando os dados de entrada(teste)

## 5.5. Previsão com Regressão Linear

In [ ]:
#treinamento usando regressão linear
lr = linear_model.LinearRegression()
lr.fit(X_train_scale, y_train)
pred_lr= lr.predict(X_test_scale)
cd =r2_score(y_test, pred_lr)
rmse_lr = sqrt(mean_squared_error(y_test, pred_lr))

print('------Linear Regression------')
print(f'Coeficiente de determinação:{cd * 100:.2f}')
print(f'RMSE: {rmse_lr}')

## 5.6. Previsão com ARIMA

In [ ]:
# Divide o dataset em 70:30 
training_data = y_train.values
test_data = y_test.values

history = [x for x in training_data]
model_predictions = []
N_test_observations = len(test_data)
for time_point in range(N_test_observations):
    model = ARIMA(history, order=(6,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = test_data[time_point]
    history.append(true_test_value)

In [ ]:
cd_arima =r2_score(test_data, np.array(model_predictions))
RMSE_error = sqrt(mean_squared_error(test_data, np.array(model_predictions)))
print('------ARIMA------')
print(f'Coeficiente de determinação: {cd_arima * 100:.2f}')
print(f'RMSE: {RMSE_error}')

## 5.7 Previsão com MLP

## 5.7.1. MLP com configuração padrão

In [ ]:
#rede neural
rn = MLPRegressor(max_iter=1000)

rn.fit(X_train_scale, y_train)
pred_rn= rn.predict(X_test_scale)

In [ ]:
cd_mlp1 = rn.score(X_test_scale, y_test)
rmse_mlp1 = sqrt(mean_squared_error(pred_rn, y_test))

print('------MLP------')
print(f'Coeficiente de determinação:{cd * 100:.2f}')
print(f'Root Mean Squared Error is {rmse_mlp1}')

## 5.7.2. MLP com configuração de ajuste de hiperparâmetros

In [ ]:
rn_mlp = MLPRegressor(max_iter=1000)

parameter_space = {
        'hidden_layer_sizes': [(i,) for i in list(range(1, 21))],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam', 'lbfgs'], 
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive'],
    }

search = GridSearchCV(rn_mlp, parameter_space, n_jobs=-1, cv=5)


search.fit(X_train_scale,y_train)
clf = search.best_estimator_
pred_mlp= search.predict(X_test_scale)

cd = search.score(X_test_scale, y_test)
rmse_mlp2 = sqrt(mean_squared_error(pred_mlp, y_test))

print(f'Coeficiente de determinação:{cd * 100:.2f}')
print(f'Root Mean Squared Error is {rmse_mlp2}')

## Extraindo resultados

Para executar a etapa de validação, será utilizado as features do dia 29/06/2021, pois os modelos foram treinados e testados com os dados históricos até dia 28/06/2021.

O objetivo é tentar prever o "Adj Close" da ação um dia a frente. No caso o dia a frente é o dia 29/06.

Portanto, para obter o resultado da validação, os modelos treinados devem obter um valor de predição próximo à "Adj Close = 30.119904"

In [ ]:
#copiando as mesmas features escolhidas
valor_novo = ultimo_registro.loc[:,['Low_dolar', 'Close_dolar', 'Close','Volume', 'High_dolar']].copy()
valor_novo

In [ ]:
#executando a predição

print("Valor real: 30.119904")

previsao=scaler.transform(valor_novo)

pred=lr.predict(previsao)
print(f'Valor predito pela Regressão Linear {pred}')

print(f'Valor predito pelo ARIMA {yhat}')

pred=rn.predict(previsao)
print(f'Valor predito pela MLP {pred}')

pred=search.predict(previsao)
print(print(f'Valor predito pela MLP Ajustado {pred}'))